### Create template target DB schema

In [10]:
%%bash
#dropdb --help

source_db="sead_bugs_import_20180503"
target_db="sead_staging_bugs_empty"

dbhost=`cat ~/.default.sead.server`
dbuser=`cat  ~/.default.sead.username`

#dbopts= --lc-collate="en_US.UTF-8" --lc-ctype="en_US.UTF-8"
# --file="sead_production_public.sql"  --format=p --no-tablespaces 

dropdb --echo --if-exists --host=$dbhost --username=$dbuser --no-password "$target_db"

createdb --echo --owner=sead_master --host=$dbhost --username=$dbuser --no-password --encoding="UTF8" "$target_db"

pg_dump --host=$dbhost --username=$dbuser --no-password --schema public --schema-only --dbname=$source_db \
    | psql --echo-errors --quiet --host=$dbhost --username=$dbuser --no-password $target_db

#pg_dump --host=$dbhost --username=$dbuser --no-password --schema bugs_import --schema-only --dbname=$source_db \
#    | psql --echo-errors --quiet --host=$dbhost --username=$dbuser --no-password $target_db

psql --echo-errors --quiet --host=$dbhost --username=$dbuser --no-password --file="01_bugs_import_schema.psql" $target_db
psql --echo-errors --quiet --host=$dbhost --username=$dbuser --no-password --file="02_setup_bugs_import_value_translations.psql" $target_db

echo "done!"


SELECT pg_catalog.set_config('search_path', '', false)
DROP DATABASE IF EXISTS sead_staging_bugs_empty;
SELECT pg_catalog.set_config('search_path', '', false)
CREATE DATABASE sead_staging_bugs_empty OWNER sead_master ENCODING 'UTF8';
 set_config 
------------
 
(1 row)

done!


ERROR:  unrecognized configuration parameter "idle_in_transaction_session_timeout"
STATEMENT:  SET idle_in_transaction_session_timeout = 0;
ERROR:  unrecognized configuration parameter "row_security"
STATEMENT:  SET row_security = off;


In [8]:
%%bash

source_db="sead_bugs_import_20180503"
target_db="sead_staging_bugs_empty"

dbhost=`cat ~/.default.sead.server`
dbuser=`cat  ~/.default.sead.username`

db_transfer_table() {
    pg_dump --host=$dbhost --username=$dbuser --no-password --table $1 --data-only --dbname=$source_db | psql --echo-errors --quiet --host=$dbhost --username=$dbuser --no-password $target_db
    
}

db_exec() {
    psql -bq -h $dbhost -U $dbuser -w -c "$1" $target_db
}

db_transfer_table "tbl_record_types"
db_transfer_table "tbl_taxa_tree_orders"
db_transfer_table "tbl_taxa_tree_families"
db_transfer_table "tbl_taxa_tree_genera"
db_transfer_table "tbl_taxa_tree_authors"
#db_transfer_table "tbl_taxonomic_order"
db_transfer_table "tbl_taxonomic_order_systems"
db_transfer_table "tbl_taxa_tree_master"

# db_exec "alter table public.tbl_taxa_tree_families alter column order_id drop not null;"


In [19]:
%%bash
psql --help

psql is the PostgreSQL interactive terminal.

Usage:
  psql [OPTION]... [DBNAME [USERNAME]]

General options:
  -c, --command=COMMAND    run only single command (SQL or internal) and exit
  -d, --dbname=DBNAME      database name to connect to (default: "roger")
  -f, --file=FILENAME      execute commands from file, then exit
  -l, --list               list available databases, then exit
  -v, --set=, --variable=NAME=VALUE
                           set psql variable NAME to VALUE
                           (e.g., -v ON_ERROR_STOP=1)
  -V, --version            output version information, then exit
  -X, --no-psqlrc          do not read startup file (~/.psqlrc)
  -1 ("one"), --single-transaction
                           execute as a single transaction (if non-interactive)
  -?, --help[=options]     show this help, then exit
      --help=commands      list backslash commands, then exit
      --help=variables     list special variables, then exit

Input and output options:
  -a, --echo-a